In [1]:
# from  import *
import importlib
import netCDF4 as nc
from math import nan

In [2]:
#NOTE! I made of copy ../add_aerosol_ics and add .py extension.
helper = importlib.import_module('add_aerosol_ics')

In [3]:
import numpy as np
try:
    import ruamel_yaml as yaml
except ImportError:
    from ruamel import yaml
from email.utils import formatdate
BlockMap = yaml.comments.CommentedMap


In [4]:
# name of original netcdf file. This script does not modify this file.
input_file="scream_unit_tests_ne2np4L72_20220822.nc"

#yaml name: it produces by the Fortran group using e3sm. This script assumes that one column of data is saved.
file_name_yaml ="aer_rad_props_sw_ts_355.yaml"

#name of output netcdf file
output_file="scream_unit_tests_aerosol_optics_ne2np4L72_20220822.nc"

# It assumes this 72 levels and state_q and qqcw have are (72x40)
#NOTE! number of variables in state_q and qqcw. This values maybe different if we use different gas chemistry
pcnst=40
nlev=72

In [5]:
src_ncdata = nc.Dataset(input_file, "r")
dst_ncdata = nc.Dataset(output_file, "w", format="NETCDF3_CLASSIC")
helper.copy_atts(src_ncdata, dst_ncdata)
# False to turn off prints
verbose=True
helper.copy_dims(src_ncdata, dst_ncdata, verbose)
helper.copy_vars(src_ncdata, dst_ncdata, verbose)

copying netCDF dimensions.
	copying ncol dimension
	copying nv dimension
	copying time dimension
	copying lev dimension
	copying dim2 dimension
	copying ilev dimension
	copying nbnd dimension
copying netCDF variables.
	copying lat variable
	copying lon variable
	copying lat_vertices variable
	copying lon_vertices variable
	copying area variable
	copying P0 variable
	copying T_mid variable
	copying T_prev_micro_step variable
	copying bm variable
	copying cldfrac_liq variable
	copying cldfrac_tot variable
	copying dim2 variable
	copying dp variable
	copying eddy_diff_mom variable
	copying eff_radius_qc variable
	copying eff_radius_qi variable
	copying h2o variable
	copying horiz_winds variable
	copying host_dx variable
	copying host_dy variable
	copying hyai variable
	copying hyam variable
	copying hybi variable
	copying hybm variable
	copying ilev variable
	copying inv_qc_relvar variable
	copying lev variable
	copying nc variable
	copying nc_nuceat_tend variable
	copying nccn variable
	

In [6]:
dst_ncdata.setncatts(
        {
            "source_file": input_file,
            "additional_history": "aerosol data added by eamxx-mam4xx add_aerosol_ics.py",
        })


In [7]:
datatype = "float64"
scalar_2d_dims = ("time", "ncol", "lev")

In [8]:
# load data from yaml files
with open(file_name_yaml, 'r') as stream:
    data_loaded = yaml.safe_load(stream)
inputs = data_loaded['input']['fixed']

In [9]:
# NOTE! state q is part of dryatm, but eamxx uses wet_atm, so we need to convert from dry to wet
# NOTE! in the couple code we will convert again from wet to dry.
state_q = np.array(inputs["state_q"]).reshape((nlev,pcnst),order='F')
# NOTE! qqcw is saved with C layout by the fortran team
qqcw = np.array(inputs["qqcw"]).reshape((nlev,pcnst), order='C')

In [10]:
def calculate_wetmmr_from_drymmr(drymmr,qv_wet):
# Note: emaxx qv_wet
    return drymmr*(1.-qv_wet)

def calculate_drymmr_from_wetmmr(wetmmr, qv_wet):
  return wetmmr/(1.-qv_wet)

def calculate_qv_wetmmr_from_qv_drymmr(qv_wet):
  return qv_wet/(1.+qv_wet)

In [11]:
qv_dry = state_q[:,0] # Note.dry water vapor mixing ratio
qv_wet = calculate_qv_wetmmr_from_qv_drymmr(qv_dry)

state_q_wet =calculate_wetmmr_from_drymmr(state_q,qv_wet.reshape((nlev,1)))
qqcw_wet =calculate_wetmmr_from_drymmr(qqcw,qv_wet.reshape((nlev,1)))

In [12]:
# these variables do not exit in original file from eamxx
var_list ={}
stateq_info = {'name':'state_q', 'shape':(nlev,pcnst),'data_yaml':qqcw_wet}
# # cldn_info = {'name':'cldn', 'shape':(nlev)}
qqcw_info = {'name':'qqcw', 'shape':(nlev,pcnst),'data_yaml':state_q_wet}
## qv
qv_wet_info = {'name':'qv_wet', 'shape':(nlev),'data_yaml':qv_wet}

var_list.update({'state_q':stateq_info})
var_list.update({'qv_wet':qv_wet_info})
var_list.update({'qqcw':qqcw_info})

In [13]:
# state_test= calculate_drymmr_from_wetmmr(state_q_wet,qv_wet.reshape((nlev,1)))
# qqcw_test= calculate_drymmr_from_wetmmr(qqcw_wet,qv_wet.reshape((nlev,1)))

In [14]:
# qqcw_test==qqcw

In [15]:
# variables exit in base nc file, but I want to update their values with values from yaml file.
update_var_dics = {}
# var# : id of variable; nc: name of variables in netcdf file; mam4: name of variables in yaml files
update_var_dics.update({'var1':{'nc':'T_mid','mam4':'temperature', 'shape':(nlev)}})
update_var_dics.update({'var2':{'nc':'z_int','mam4':'zi', 'shape':(nlev+1)}})
update_var_dics.update({'var3':{'nc':'z_mid','mam4':'zm', 'shape':(nlev)}})
update_var_dics.update({'var4':{'nc':'p_mid','mam4':'pmid', 'shape':(nlev)}})
update_var_dics.update({'var5':{'nc':'p_int','mam4':'pint', 'shape':(nlev+1)}})
update_var_dics.update({'var6':{'nc':'pseudo_density','mam4':'pdel', 'shape':(nlev)}})
update_var_dics.update({'var7':{'nc':'pseudo_density_dry','mam4':'pdeldry', 'shape':(nlev)}})
# NOTE!: here I would like to update values of cldfrac_tot
update_var_dics.update({'var8':{'nc':'cldfrac_tot','mam4':'cldn', 'shape':(nlev)}})


In [16]:
for ivar in update_var_dics:
  yaml_name = update_var_dics[ivar]['mam4']
  var_1d = np.array(inputs[yaml_name])
  var_nd = var_1d.reshape(update_var_dics[ivar]['shape'], order='F')
  nc_name =  update_var_dics[ivar]['nc']
  var= dst_ncdata.variables[nc_name]
  # Note: we assume that all columns have same values
  var[0,:,:] = var_nd


In [17]:
# state_q(:,1) is used in several places in the fortran code.
# according with code 1 is h2o, which corresponds to 0 in python
update_var_dics_post_computed={}
update_var_dics_post_computed.update({'var0':{'nc':'qv',"values":qv_wet, 'shape':(nlev),'idx':-1}})

add_these_vars =['qc','qi','nc','ni']
# order is important
for idx,var in enumerate(add_these_vars,1):
  update_var_dics_post_computed.update({'var'+str(idx):{'nc':var,"values":state_q_wet[:,idx],
                                                'shape':(nlev)}})
for ivar in update_var_dics_post_computed:
  nc_name =  update_var_dics_post_computed[ivar]['nc']
  try:
    # create variables if it does not exit in original nc file
    save_var = dst_ncdata.createVariable(
        nc_name, datatype, scalar_2d_dims, fill_value=nan)
    save_var.setncatts(
        {
            "mdims": 1,
            "units": "kg/kg",
            "long_name": nc_name,
        }
      )
  except:
      # variable name exits in original nc file
      save_var = dst_ncdata.variables[nc_name]
      # var= dst_ncdata.variables[nc_name]
  # Note: we assume that all columns have same values
  save_var[0,:,:] = update_var_dics_post_computed[ivar]["values"]


In [18]:
# map to index from state_q to vmr
map2chm = [         0,         0,         0,         0,         0,         0,         0,         0,         0,         1,         2,         3,         4,         5,         6,         7,         8,         9,        10,        11,        12,        13,        14,        15,        16,        17,        18,        19,        20,        21,        22,        23,        24,        25,        26,        27,        28,        29,        30,        31]

In [19]:
#this list is produce by chemistry pre-process
#Note : original name in e3sm is ncl. I change this name to nacl because we use nacl in eamxx
solsym =[ 'O3              ','H2O2            ','H2SO4           ','SO2             ','DMS             ',
                        'SOAG            ','so4_a1          ','pom_a1          ','soa_a1          ','bc_a1           ',
                        'dst_a1          ','nacl_a1          ','mom_a1          ','num_a1          ','so4_a2          ',
                        'soa_a2          ','nacl_a2          ','mom_a2          ','num_a2          ','dst_a3          ',
                        'nacl_a3          ','so4_a3          ','bc_a3           ','pom_a3          ','soa_a3          ',
                        'mom_a3          ','num_a3          ','pom_a4          ','bc_a4           ','mom_a4          ',
                        'num_a4          ']
var_names_vmr =[]
# delete white space
for ivar in solsym:
  var_names_vmr += [ivar.replace(" ", "")]

In [20]:
update_var_dics_prog={}
for mm in range(pcnst):
  nn = map2chm[mm]-1
  if( nn > -1 ):
    # mmr(:ncol,:,nn) = state_q(:ncol,:,mm)
    # print(mm,nn,var_names_vmr[nn])
    # asummes that name of aerosol species is NAME_a.
    cld_varname = var_names_vmr[nn].replace('_a', '_c')
    if cld_varname==var_names_vmr[nn]:
       #gas specie
       # Note: we use lower case for gas species names in eamxx
      update_var_dics_prog.update({'var'+str(mm):{'nc':var_names_vmr[nn],#.lower()
                                                    'mam4':'state_q', 'shape':(nlev),'idx':mm}})
    else:
      # aerosol
      update_var_dics_prog.update({'var_a'+str(mm):{'nc':var_names_vmr[nn],
                                                    'mam4':'state_q', 'shape':(nlev),'idx':mm}})
      update_var_dics_prog.update({'var_c'+str(mm):{'nc':cld_varname,
                                                    'mam4':'qqcw', 'shape':(nlev),'idx':mm}})

In [21]:
for ivar in update_var_dics_prog:
  yaml_name = update_var_dics_prog[ivar]['mam4']
  var_2d = var_list[yaml_name]['data_yaml']
  nc_name =  update_var_dics_prog[ivar]['nc']
  try:
    # create variables if it does not exit in original nc file
    save_var = dst_ncdata.createVariable(
        nc_name, datatype, scalar_2d_dims, fill_value=nan)
    save_var.setncatts(
        {
            "mdims": 1,
            "units": "kg/kg",
            "long_name": yaml_name +" from e3sm with idx "+ str(update_var_dics_prog[ivar]['idx']),
        }
      )
  except:
      # variable name exits in original nc file
      save_var = dst_ncdata.variables[nc_name]
  save_var[0,:]= var_2d[:, update_var_dics_prog[ivar]['idx']]

In [22]:
dst_ncdata.close()
